In [1]:
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"  


# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# import torch

# CKPT = "./outputs/en2ko-nllb600m-lora_qv_all/checkpoint-962"   # 마지막 또는 best
# tok = AutoTokenizer.from_pretrained(CKPT)
# model = AutoModelForSeq2SeqLM.from_pretrained(CKPT,use_safetensors=True ).to("cuda").eval()

# tok.src_lang = "eng_Latn"
# model.config.forced_bos_token_id = tok.convert_tokens_to_ids("kor_Hang")

In [5]:
# text = "I can't believe you said that!"
# ids = tok(text, return_tensors="pt", truncation=True, max_length=128).to("cuda")
# out = model.generate(**ids, max_new_tokens=128, num_beams=4)
# print("SRC :", text)
# print("HYP :", tok.decode(out[0], skip_special_tokens=True))


SRC : I can't believe you said that!
HYP : 네가 말한 게 믿기지 않아!


In [7]:
import pandas as pd
from tqdm import tqdm

df = pd.read_csv("./popsongData/parsed_en2ko_test.csv")  # 테스트셋
samples = df.sample(50, random_state=42)   # 일부만

hyps = []
for s in tqdm(samples["en"]):
    ids = tok(s, return_tensors="pt", truncation=True, max_length=128).to("cuda")
    out = model.generate(**ids, max_new_tokens=128, num_beams=4)
    hyps.append(tok.decode(out[0], skip_special_tokens=True))

for src, hyp, ref in zip(samples["en"], hyps, samples["ko"]):
    print("SRC:", src)
    print("HYP:", hyp)
    print("REF:", ref)
    print()


100%|██████████| 50/50 [00:08<00:00,  6.23it/s]

SRC: Daily duckin lames
HYP: 매일 duckin lames
REF: 매일 바보들을 피해

SRC: Redbone suckin' on me, let me bust my nut then switch the lanes
HYP: 붉은 뼈가 날 빨아들여, 너트를 깨고 차선을 바꿀게
REF: 나를 빨아대는 예쁜 여자들, 사정부터 하고 길을 바꾸지

SRC: I don't want none of that extra loud shit
HYP: 별다른 시끄러운 건 원하지 않아
REF: 특히 독한 것들 딱히 원하지 않아

SRC: Okay, okay, okay, okay
HYP: 그래, 그래, 그래, 그래
REF: Okay, okay, okay, okay

SRC: But I was born with it, it's natural, word to mommy (Uhu)
HYP: 하지만 난 타고난 거야, 당연한 거야, 엄마한테 한 말 (Uhu)
REF: 허나 갖고 태어난 거지, 타고 난 거야, 엄마를 걸고 (Uhu)

SRC: I will respect and love you
HYP: 널 존중하고 사랑할 거야
REF: 널 존중하고 사랑할 거야

SRC: Don't wanna hurt you (thank you)
HYP: 널 다치게 하고 싶지 않아 (고맙네)
REF: 널 다치게 하고 싶지 않아 (않아)

SRC: In a grave within a grave, it was the first time I prayed
HYP: 무덤 안의 무덤에서, 기도한 건 처음이었어
REF: 무덤 안의 무덤 안에서, 처음으로 나는 기도했지

SRC: The earth shift, the kiss of death, here’s the reaper to flirt with
HYP: 지구 변화, 죽음의 키스, 여기 플리팅할 리퍼가 있어
REF: 지구가 흔들리게, 죽음의 키스, 저승사자를 꼬셔봐

SRC: Roman numerals in my watch, leather Corinthia

In [24]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  


# ==== 0) 경로/체크포인트 ====
CSV_PATH = "./SpotifyLyrics/lyrics_0414.csv"   # ← 당신의 테스트 CSV 경로
CKPT     = "./outputs/en2ko-nllb600m-lora-other-parms4/checkpoint-3000"      # ← 마지막(또는 best) 체크포인트로 교체

# ==== 1) 라이브러리/모델 로드 ====
import os, torch, pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

os.environ["TOKENIZERS_PARALLELISM"] = "false"
device = "cuda" if torch.cuda.is_available() else "cpu"

tok = AutoTokenizer.from_pretrained(CKPT)
model = AutoModelForSeq2SeqLM.from_pretrained(CKPT,use_safetensors=True).to(device).eval()

# NLLB 언어 코드
tok.src_lang = "eng_Latn"
model.config.forced_bos_token_id = tok.convert_tokens_to_ids("kor_Hang")

# ==== 2) 데이터 로드 ====
df = pd.read_csv(CSV_PATH, encoding="cp949")  # ← ansi면 cp949 시도
# df = pd.read_csv(CSV_PATH, encoding="euc-kr")  # 혹은 이걸로
# df = pd.read_csv(CSV_PATH, encoding="etf-8")  # 혹은 이걸로

# 컬럼명 정규화: en/ko가 기본, 아니면 추정해서 맞춤
if "en" not in df.columns:
    if "English" in df.columns: df = df.rename(columns={"English":"en"})
    elif "src" in df.columns:   df = df.rename(columns={"src":"en"})
if "ko" not in df.columns:
    if "Korean" in df.columns:  df = df.rename(columns={"Korean":"ko"})
    elif "tgt" in df.columns:   df = df.rename(columns={"tgt":"ko"})

assert "en" in df.columns, "입력 컬럼(en/english/src)이 필요합니다."
# ko(레퍼런스)는 없어도 됨(없으면 비교만 생략)
has_ref = "ko" in df.columns

# ==== 3) 배치 생성 함수 ====
def translate_list(texts, max_src=128, max_new=128, beams=1, batch=64):
    outs = []
    for i in tqdm(range(0, len(texts), batch), desc="Generating"):
        chunk = texts[i:i+batch]
        ids = tok(chunk, return_tensors="pt", truncation=True, max_length=max_src, padding=True).to(device)
        with torch.no_grad():
            gen = model.generate(
                **ids,
                max_new_tokens=max_new,
                num_beams=beams,
                do_sample=False
            )
        outs.extend(tok.batch_decode(gen, skip_special_tokens=True))
        del ids, gen
        torch.cuda.empty_cache()
    return outs

# ==== 4) 추론 실행 (빠른 확인은 beams=1, 품질확인은 4) ====
HYP = translate_list(df["en"].tolist(), max_src=128, max_new=128, beams=4, batch=64)
df["hyp"] = HYP

# ==== 5) 미리보기 & 저장 ====
cols = ["en", "hyp"] + (["ko"] if has_ref else [])
display(df[cols].head(30))  # Jupyter 미리보기
out_path = os.path.splitext(CSV_PATH)[0] + "_with_p4_mt_3000_beams_4.csv"
df[cols].to_csv(out_path, index=False, encoding="utf-8-sig")
print("Saved ->", out_path)

# ==== 6) (옵션) BLEU 정량 평가 ====
try:
    if has_ref:
        import evaluate
        metric = evaluate.load("sacrebleu")
        score = metric.compute(predictions=df["hyp"].tolist(),
                               references=[[r] for r in df["ko"].fillna("").tolist()])
        print("sacreBLEU:", score["score"])
except Exception as e:
    print("BLEU 스킵:", e)


Generating: 100%|██████████| 2/2 [00:05<00:00,  2.67s/it]


,en,hyp,ko
0,Turned around and said goodbye,돌아서 작별 인사를 했지,NaN
1,"Use your lips to manipulate, uh","네 입술을 이용해 조작해, 어",NaN
2,You start fights while we're up late,넌 우리가 늦게 일어나는 동안 싸움을 시작해,NaN
3,'Cause if I won it all without you involved (f...,네가 관여하지 않고 내가 모든 걸 이겼다면 (젠장 사랑해),NaN
4,"(Mmm, ah-ah) been waiting on your call all day","(엠엠, 아-아) 하루 종일 네 전화를 기다렸어",NaN
5,"(Mmm, ah-ah) I wonder where I lost my way?","(흠, 아-아) 어디로 길을 잃었는지 궁금해?",NaN
6,Ain't no savin' face this time,이번에는 면모가 없어,NaN
7,"And if it's up, stay down for me, yeah","만약 올라갔다면, 나를 위해 내려앉아 있어, 그래",NaN
8,"Baby, you make me pray for London, yeah","베이비, 넌 날 런던을 위해 기도하게 해, 그래",NaN
9,But my past won't keep me from my best (I-I-I-...,하지만 내 과거가 나를 최선을 다하는 데 방해하지 않을 거야 (난-난-난-난-난-난...,NaN


Saved -> ./SpotifyLyrics/lyrics_0414_with_p4_mt_3000_beams_4.csv
sacreBLEU: 0.0
